# Atlas Processing - Initial Image Creation

## OTS labels - Bouhali, Wiener

- Documentation: https://nilearn.github.io/auto_examples/01_plotting/plot_surf_atlas.html
- Note: nilearn's 'surface.load_surface_data' function does not suppport .mgh / .mgz files; must use mri_convert or mris_convert (part of Freesurfer) to first convert to an acceptable format, e.g. .nii

In [2]:
import nilearn
from nilearn import surface
from nilearn import plotting

import os
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
# import png  # for reloading / working with previously saved images

/home/groups/jyeatman/software/anaconda3/envs/torch-ni-ny/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


## Set up directories

In [3]:
# === DIRECTORIES === #

# input data directories
# overall format:
# -- hbn_dir/sub-{EID}/label_subdir/parc_filename

hbn_dir = '/scratch/users/samjohns/HBN/BIDS_curated/derivatives/freesurfer'
label_subdir = '/label'
surf_subdir = '/surf'

curv_filename = 'lh.curv'
infl_filename = 'lh.inflated'
pial_filename = 'lh.pial'
parc_filename = 'lh.aparc.a2009s.annot'

# output data directories
out_data_dir = '/scratch/groups/jyeatman/samjohns-projects/data/atlas'
out_parc = out_data_dir + '/parc'
out_curv = out_data_dir + '/curv'
os.makedirs(out_parc, exist_ok=True)  # ensure image output directory exists
os.makedirs(out_curv, exist_ok=True)  # ensure image output directory exists
assert os.path.exists(out_curv)

# === LABELS === #

# important:
# select a subset of labels that are visible in  ventral view
# label = 43 was borderline and removed for convenience
labels_to_plot = [2, 19, 21, 23, 24, 25, 30, 37, 38, 50, 51, 57, 58, 59, 60, 61, 63, 65]

In [4]:
fname = 'subjects-to-atlas.txt'
subjects = [s for s in os.listdir(hbn_dir) if 'sub-' in s]
subjects.sort()
with open(fname, 'w') as f:
    for s in subjects:
        f.write(s + '\n')

In [5]:
subjects = subjects[:10]

## Make images

In [6]:
curv_mask_dir = '/scratch/groups/jyeatman/samjohns-projects/data/curv-mask-jitter'
mask_fns = os.listdir(curv_mask_dir)

In [7]:
def get_angle(s):
    angles = s.split('-')[2:4]
    angles = [float(a) for a in angles]
    return angles

def get_eid(s):
    return 'sub-' + s.split('-')[1]

In [8]:
fn_df = pd.DataFrame({'Filename': mask_fns})
fn_df['Angles'] = fn_df['Filename'].apply(get_angle)
fn_df['EID'] = fn_df['Filename'].apply(get_eid)

In [9]:
fn_df.sort_values('EID', inplace=True)

In [10]:
subjects = list(fn_df['EID'].unique())

In [11]:
len(subjects)

2361

In [12]:
def make_subject_images(sub, angles):
    parc_path = f'{hbn_dir}/{sub}{label_subdir}/{parc_filename}'
    curv_path = f'{hbn_dir}/{sub}{surf_subdir}/{curv_filename}'
    infl_path = f'{hbn_dir}/{sub}{surf_subdir}/{infl_filename}'
    
    # check files exist
    if not (os.path.exists(parc_path) 
            and os.path.exists(curv_path) 
            and os.path.exists(infl_path)
           ):
        return

    parc = surface.load_surf_data(parc_path)
    curv = surface.load_surf_data(curv_path)
    infl = surface.load_surf_mesh(infl_path)
    
    selected_parc = np.array([labels_to_plot.index(l) if l in labels_to_plot else -1 for l in parc])
    
    for a in angles:

        fig, ax = plt.subplots(figsize=(8, 8))
        plotting.plot_surf_roi(infl, selected_parc
                               ,view=(a[0], a[1])
                               # ,bg_map=test_curv
                               # ,bg_on_data=True 
                               ,figure=fig
                               ,cmap='tab20'
                               ,output_file=f'{out_parc}/{sub}-{a[0]}-{a[1]}-parc.png'   
                               # ,threshold=25.0
                               # colorbar=True
                              )

        fig, ax = plt.subplots(figsize=(8, 8))
        plotting.plot_surf_roi(infl, selected_parc
                               ,view=(a[0], a[1])
                               ,bg_map=curv
                               # ,bg_on_data=True 
                               ,figure=fig
                               ,cmap='tab20'
                               ,threshold=25.0
                               ,output_file=f'{out_curv}/{sub}-{a[0]}-{a[1]}-curv.png'
                               # colorbar=True
                              )

In [60]:
done_curv_fn = os.listdir(out_curv)
done_df = pd.DataFrame({'Filename': done_curv_fn})
done_df['Angles'] = done_df['Filename'].apply(get_angle)
done_df['EID'] = done_df['Filename'].apply(get_eid)

counts = done_df.groupby('EID', as_index=False).count()
counts['Count'] = counts['Angles']
del counts['Angles']
counts = counts[['EID', 'Count']]

done_subjects = list(counts[counts.Count==40]['EID'])
subjects_todo = list(set(subjects) - set(done_subjects))

In [61]:
len(subjects_todo)

772

In [ ]:
# main loop: don't execute this cell unless you want to go grab a coffee
for sub in subjects_todo[:1]:
    angles = list(fn_df[fn_df.EID==sub]['Angles'])
    make_subject_images(sub, angles)